In [1]:
import pandas as pd
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 500)

# Process Outline
* create a number of identical starting databases 
  * (based on current prodonprem and slightly out of date data)
* for database in databases:
  * switch database
  * download tables
  * run migration
  * download tables
  * run script
  * download tables


### Attempt 1: dmapps_test
* mostly worked - FY error (eg, Apr 22 is FY 23)
* 5 requests that changed status after saving - investigate more deeply

In [2]:
requests = 'dmapps_test_csas2_csasrequest.csv'
tor = 'dmapps_test_csas2_termsofreference.csv'

r1_before = pd.read_csv('before/'+requests)
r1_after = pd.read_csv('after/'+requests)
r1_after_save = pd.read_csv('after_saving/'+requests)
tor1_before = pd.read_csv('before/'+tor)
tor1_after = pd.read_csv('after/'+tor)
tor1_after_save = pd.read_csv('after_saving/'+tor)

##### tor

In [3]:
print(tor1_before.compare(tor1_after).shape)
tor1_before.compare(tor1_after).fillna('').head()

(207, 4)


updated_at                             submission_date  \
                          self                       other            self   
31  2023-01-27 16:13:03.828064  2024-01-12 13:09:18.020844                   
51  2023-01-27 15:54:22.721896  2024-01-12 13:09:18.024822                   
64  2023-01-27 16:15:46.300804  2024-01-12 13:09:18.029821                   
80  2023-01-27 16:23:19.365783  2024-01-12 13:09:18.035340                   
83  2023-01-27 16:07:14.774777  2024-01-12 13:09:18.040531                   

                         
                  other  
31  2022-04-01 12:00:00  
51  2022-04-01 12:00:00  
64  2023-04-01 12:00:00  
80  2023-04-01 12:00:00  
83  2023-04-01 12:00:00

In [4]:
print(tor1_after.compare(tor1_after_save).shape)
tor1_after.compare(tor1_after_save).fillna('').head()

(527, 2)


updated_at                            
                         self                       other
0  2021-12-16 20:24:42.921503  2024-01-12 13:11:18.733992
1  2021-12-16 20:24:40.716634  2024-01-12 13:11:18.794738
2  2021-12-16 20:24:41.093677  2024-01-12 13:11:18.868574
3  2021-12-16 20:24:42.538060  2024-01-12 13:11:18.972617
4  2021-12-16 20:24:41.847219  2024-01-12 13:11:19.011850

In [5]:
tor1_after.shape

(527, 19)

##### requests

In [6]:
print(r1_before.compare(r1_after).shape)
r1_before.compare(r1_after).fillna('').head()

(187, 4)


updated_at                             submission_date  \
                           self                       other            self   
68   2022-08-25 22:30:01.944633  2024-01-12 13:09:17.975603                   
69   2022-08-25 22:30:03.580007  2024-01-12 13:09:17.979603                   
74   2022-08-25 22:30:02.953676  2024-01-12 13:09:17.976604                   
75   2022-08-25 22:30:03.379908  2024-01-12 13:09:17.977603                   
104  2023-03-14 17:45:46.225119  2024-01-12 13:09:17.980602                   

                          
                   other  
68   2022-04-01 12:00:00  
69   2022-04-01 12:00:00  
74   2022-04-01 12:00:00  
75   2022-04-01 12:00:00  
104  2022-04-01 12:00:00

In [7]:
print(r1_after.compare(r1_after_save).shape)
r1_after.drop('updated_at', axis=1).compare(r1_after_save.drop('updated_at', axis=1)).fillna('').head(15)

(1198, 6)


status       fiscal_year_id        
       self other           self   other
10     70.0  41.0                       
68                        2022.0  2023.0
69                        2022.0  2023.0
74                        2022.0  2023.0
75                        2022.0  2023.0
104                       2022.0  2023.0
214    70.0  41.0                       
256    70.0  41.0                       
766    41.0  70.0                       
791    20.0  30.0                       
1010                      2010.0  2011.0
1011                      2017.0  2018.0
1012                      2015.0  2016.0
1013                      2018.0  2019.0
1014                      2019.0  2020.0

In [8]:
r1_after.drop('updated_at', axis=1).compare(r1_after_save.drop('updated_at', axis=1)).fillna('').tail()

status       fiscal_year_id        
       self other           self   other
1187                      2022.0  2023.0
1188                      2021.0  2022.0
1189                      2022.0  2023.0
1190                      2022.0  2023.0
1191                      2022.0  2023.0

In [9]:
r1_after_save.iloc[1191]

id                                                                     1349
uuid                                       f9149f19e3d24241b2daa4f5fc88a103
created_at                                       2023-05-08 12:56:53.217275
updated_at                                       2024-01-12 13:11:45.896215
language                                                                  1
title                     Biofouling as a Vector for Aquatic Invasive Sp...
is_multiregional                                                        0.0
multiregional_text                                                      NaN
issue                                                                   NaN
assistance_text                                                         NaN
rationale                                                               NaN
risk_text                                                               NaN
advice_needed_by                                        2022-01-10 16:00:00
rationale_fo

In [10]:
# OK, it looks like FY 2023 is Apr 22 to Mar 23, so the migration needs to be adjusted...
# let's confirm that this is the correct assumption by counting errors (should be 187)

print(r1_after.drop([
    'updated_at', 'status'
], axis=1).compare(r1_after_save.drop([
    'updated_at', 'status'
], axis=1)).shape)

(187, 2)


In [11]:
# other messed up things that aren't FY related

display(
    r1_after.drop([
        'updated_at', 'fiscal_year_id'
    ], axis=1).compare(r1_after_save.drop([
        'updated_at', 'fiscal_year_id'
    ], axis=1))    
)

request_problems = r1_after.drop([
    'updated_at', 'fiscal_year_id'
], axis=1).compare(r1_after_save.drop([
    'updated_at', 'fiscal_year_id'
], axis=1)).index

r1_after.loc[request_problems]

status      
      self other
10    70.0  41.0
214   70.0  41.0
256   70.0  41.0
766   41.0  70.0
791   20.0  30.0

,id,uuid,created_at,updated_at,language,title,is_multiregional,multiregional_text,issue,assistance_text,rationale,risk_text,advice_needed_by,rationale_for_timeline,has_funding,funding_text,prioritization,prioritization_text,status,submission_date,old_id,client_id,created_by_id,section_id,updated_by_id,translated_title,ref_number,advice_fiscal_year_id,fiscal_year_id,office_id
10,12,d5f44c1425824bf0beaddbd00a0adcad,2021-04-28 10:37:53.728292,2022-08-25 22:30:25.223383,1,Science Response - Striped bass SGSL,0.0,NaN,1. Stock status update\r\n1. Establishing an u...,NaN,The results of this science advice will suppor...,"If no science response is provided, SDO will n...",2018-11-01 03:00:00,The science advice will prepare DFO for the de...,0,NaN,NaN,NaN,70,2021-04-28 10:42:19.913898,NaN,1802,50.0,117,50.0,NaN,GLF-2018-2019-TAB07,2021,2022,1
214,252,c1a9258bc16f40cfaa47d08dcf2107ce,2021-07-12 16:15:36.340549,2023-04-06 16:09:37.455726,1,2018SAL07 Fraser Sockeye Post Season Run Size ...,0.0,"Fisheries Management, Science, Pacific Salmon ...",Review of and documentation of DFO methods cur...,"Jamie Scroggie, Jennifer Nener, David Patterson",The results will impact DFO programs in the Pa...,If the review is not completed there will be s...,2022-12-31 04:00:00,Data is required for annual abundance forecast...,0,NaN,3.0,NaN,70,2021-12-16 12:00:00,NaN,1952,1555.0,139,1555.0,NaN,2018SAL07,2023,2022,5
256,295,802464b095c34ba1ac32bfebd6afc606,2021-07-14 23:44:19.318855,2023-04-06 16:08:27.033001,1,2018SAR12 Recovery Potential Assessment (RPA) ...,0.0,NaN,Using the Guidance for the Completion of Recov...,"Paul Grant, Research Scientist",The COSEWIC assessment for Fraser River sockey...,"If the RPA is not completed, subsequent compon...",2022-03-31 03:00:00,"After the COSEWIC assessment is complete, DFO ...",1,Funds are requested through the annual SARA wo...,1.0,NaN,70,2021-12-16 12:00:00,NaN,1968,1555.0,142,1500.0,NaN,2018SAR12,2022,2022,5
766,904,a76a8adb681b478eb84c439dc212f378,2022-10-21 11:47:56.786293,2022-12-01 15:46:36.583871,1,Updated Biological Reference Points for Gasper...,1.0,Maritimes - Resource Management - Management o...,FFHPP and Resource Management are seeking DFO ...,FFHPP and Resource Management staff have discu...,Existing reference points for managing the Gas...,"If Scientific advice is not provided, then the...",2023-08-31 03:00:00,"To allow time to complete the analysis, draft ...",0,NaN,1.0,Existing reference points are believed to be i...,41,2022-10-27 13:48:46.212584,NaN,778,778.0,124,778.0,NaN,NaN,2024,2023,2
791,930,cae2109c01b24a64a538ba5118777e1b,2022-10-25 19:44:26.256159,2022-11-04 15:49:40.098824,1,Assessment of fishery and non-fishery interact...,1.0,The range of the NW Atlantic population of Lea...,"In March 2018, a CSAS Regional Science Respons...",We have discussed the feasibility of this requ...,The advice resulting from this request will su...,This science advice would help us meet a commi...,2026-03-31 03:00:00,To synchronize and meet the commitments mentio...,0,Species at Risk Program funds,1.0,Understanding the sources and rates of interac...,20,2022-10-26 17:39:16.478281,NaN,947,947.0,125,947.0,NaN,NaN,2026,2023,2


### Attempt 2: dmapps_test2
* worked almost perfectly
* 5 statuses changed after saving

In [12]:
requests2 = 'dmapps_test2_csas2_csasrequest.csv'
tor2 = 'dmapps_test2_csas2_termsofreference.csv'

r2_before = pd.read_csv('before/'+requests2)
r2_after = pd.read_csv('after/'+requests2)
r2_after_save = pd.read_csv('after_saving/'+requests2)
tor2_before = pd.read_csv('before/'+tor2)
tor2_after = pd.read_csv('after/'+tor2)
tor2_after_save = pd.read_csv('after_saving/'+tor2)

##### tor

In [13]:
print(tor2_before.compare(tor2_after).shape)
tor2_before.compare(tor2_after).fillna('').head()

(207, 4)


updated_at                             submission_date  \
                          self                       other            self   
31  2023-01-27 16:13:03.828064  2024-01-12 14:16:04.532632                   
51  2023-01-27 15:54:22.721896  2024-01-12 14:16:04.539554                   
64  2023-01-27 16:15:46.300804  2024-01-12 14:16:04.547355                   
80  2023-01-27 16:23:19.365783  2024-01-12 14:16:04.554935                   
83  2023-01-27 16:07:14.774777  2024-01-12 14:16:04.561838                   

                         
                  other  
31  2022-04-01 12:00:00  
51  2022-04-01 12:00:00  
64  2023-04-01 12:00:00  
80  2023-04-01 12:00:00  
83  2023-04-01 12:00:00

In [14]:
print(tor2_after.compare(tor2_after_save).shape)
tor2_after.compare(tor2_after_save).fillna('').head()

(527, 2)


updated_at                            
                         self                       other
0  2021-12-16 20:24:42.921503  2024-01-12 14:20:15.078247
1  2021-12-16 20:24:40.716634  2024-01-12 14:20:15.153871
2  2021-12-16 20:24:41.093677  2024-01-12 14:20:15.209887
3  2021-12-16 20:24:42.538060  2024-01-12 14:20:15.412313
4  2021-12-16 20:24:41.847219  2024-01-12 14:20:15.464743

##### requests

In [15]:
print(r2_before.compare(r2_after).shape)
r2_before.compare(r2_after).fillna('').head()

(187, 4)


updated_at                             submission_date  \
                           self                       other            self   
68   2022-08-25 22:30:01.944633  2024-01-12 14:16:04.239687                   
69   2022-08-25 22:30:03.580007  2024-01-12 14:16:04.247726                   
74   2022-08-25 22:30:02.953676  2024-01-12 14:16:04.242278                   
75   2022-08-25 22:30:03.379908  2024-01-12 14:16:04.245137                   
104  2023-03-14 17:45:46.225119  2024-01-12 14:16:04.250635                   

                          
                   other  
68   2021-04-01 12:00:00  
69   2021-04-01 12:00:00  
74   2021-04-01 12:00:00  
75   2021-04-01 12:00:00  
104  2021-04-01 12:00:00

In [16]:
print(r2_after.drop('updated_at', axis=1).compare(r2_after_save.drop('updated_at', axis=1)).shape)
r2_after.drop('updated_at', axis=1).compare(r2_after_save.drop('updated_at', axis=1)).fillna('')

(5, 2)


status      
      self other
10    70.0  41.0
214   70.0  41.0
256   70.0  41.0
766   41.0  70.0
791   20.0  30.0

### Why do these statuses change?

In [17]:
pd.merge(
    r2_before.loc[request_problems, ['id', 'status']], 
    r2_after_save.loc[request_problems, ['id', 'status']],
    on='id',
    suffixes=('_before', '_after')
)

,id,status_before,status_after
0,12,70,41
1,252,70,41
2,295,70,41
3,904,41,70
4,930,20,30


* 12, 252, and 295 must not have a process
  * CONFIRMED CORRECT (SQL below)
* 904 must have a process
  * CONFIRMED CORRECT (SQL below)
* 930 must have had a completed review?
  * CONFIRMED ALL REVIEWS ARE COMPLETE (via app, saving updated the status, even in prodonprem branch with no changes to def save method)

```SQL
SELECT csas2_process.id AS p_id, csas2_csasrequest.id AS r_id FROM csas2_csasrequest
LEFT JOIN csas2_process_csas_requests ON csas2_process_csas_requests.csasrequest_id = csas2_csasrequest.id  
LEFT JOIN csas2_process ON csas2_process_csas_requests.process_id = csas2_process.id 
WHERE csas2_csasrequest.id IN (12, 252, 295, 904, 930)
```

In [18]:
# all of the problem status requests have submission dates

r1_before.loc[request_problems, 'submission_date']

10     2021-04-28 10:42:19.913898
214           2021-12-16 12:00:00
256           2021-12-16 12:00:00
766    2022-10-27 13:48:46.212584
791    2022-10-26 17:39:16.478281
Name: submission_date, dtype: object

### Attempt 3: Confirm That Everything Works (using dmapps_test3)
* didn't do after case, just before and after_save

In [19]:
requests3 = 'dmapps_test3_csas2_csasrequest.csv'
tor3 = 'dmapps_test3_csas2_termsofreference.csv'

r3_before = pd.read_csv('before/'+requests3)
r3_after_save = pd.read_csv('after_saving/'+requests3)
tor3_before = pd.read_csv('before/'+tor3)
tor3_after_save = pd.read_csv('after_saving/'+tor3)

##### tor

In [20]:
print(tor3_before.drop('updated_at', axis=1).compare(tor3_after_save.drop('updated_at', axis=1)).shape)
tor3_before.drop('updated_at', axis=1).compare(tor3_after_save.drop('updated_at', axis=1)).fillna('').value_counts()

(207, 2)


(submission_date, self)  (submission_date, other)
                         2020-04-01 12:00:00         32
                         2021-04-01 12:00:00         32
                         2022-04-01 12:00:00         28
                         2019-04-01 12:00:00         23
                         2017-04-01 12:00:00         16
                         2018-04-01 12:00:00         16
                         2023-04-01 12:00:00         14
                         2015-04-01 12:00:00         12
                         2016-04-01 12:00:00         12
                         2012-04-01 12:00:00          6
                         2013-04-01 12:00:00          6
                         2014-04-01 12:00:00          6
                         2011-04-01 12:00:00          3
                         2010-04-01 12:00:00          1
dtype: int64

##### requests

In [21]:
print(r3_before.drop('updated_at', axis=1).compare(r3_after_save.drop('updated_at', axis=1)).shape)
r3_before.drop('updated_at', axis=1).compare(r3_after_save.drop('updated_at', axis=1)).fillna('').value_counts()

(192, 4)


(status, self)  (status, other)  (submission_date, self)  (submission_date, other)
                                                          2019-04-01 12:00:00         32
                                                          2020-04-01 12:00:00         32
                                                          2018-04-01 12:00:00         23
                                                          2021-04-01 12:00:00         22
                                                          2016-04-01 12:00:00         16
                                                          2017-04-01 12:00:00         16
                                                          2014-04-01 12:00:00         12
                                                          2015-04-01 12:00:00         12
                                                          2011-04-01 12:00:00          6
                                                          2012-04-01 12:00:00          6
                           

### Attempt 4: Slight Update (dmapps_test5)
* after tweaking tor FY to match requests

In [22]:
requests4 = 'dmapps_test5_csas2_csasrequest.csv'
tor4 = 'dmapps_test5_csas2_termsofreference.csv'

r4_before = pd.read_csv('before/'+requests4)
r4_after = pd.read_csv('after/'+requests4)
r4_after_save = pd.read_csv('after_saving/'+requests4)
tor4_before = pd.read_csv('before/'+tor4)
tor4_after = pd.read_csv('after/'+tor4)
tor4_after_save = pd.read_csv('after_saving/'+tor4)

##### tors

In [23]:
print(tor4_before.drop('updated_at', axis=1).compare(tor4_after.drop('updated_at', axis=1)).shape)
tor4_before.drop('updated_at', axis=1).compare(tor4_after.drop('updated_at', axis=1)).fillna('').value_counts()

(207, 2)


(submission_date, self)  (submission_date, other)
                         2019-04-01 12:00:00         32
                         2020-04-01 12:00:00         32
                         2021-04-01 12:00:00         28
                         2018-04-01 12:00:00         23
                         2016-04-01 12:00:00         16
                         2017-04-01 12:00:00         16
                         2022-04-01 12:00:00         14
                         2014-04-01 12:00:00         12
                         2015-04-01 12:00:00         12
                         2011-04-01 12:00:00          6
                         2012-04-01 12:00:00          6
                         2013-04-01 12:00:00          6
                         2010-04-01 12:00:00          3
                         2009-04-01 12:00:00          1
dtype: int64

In [24]:
print(tor4_before.drop('updated_at', axis=1).compare(tor4_after_save.drop('updated_at', axis=1)).shape)
tor4_before.drop('updated_at', axis=1).compare(tor4_after_save.drop('updated_at', axis=1)).fillna('').value_counts()

(207, 2)


(submission_date, self)  (submission_date, other)
                         2019-04-01 12:00:00         32
                         2020-04-01 12:00:00         32
                         2021-04-01 12:00:00         28
                         2018-04-01 12:00:00         23
                         2016-04-01 12:00:00         16
                         2017-04-01 12:00:00         16
                         2022-04-01 12:00:00         14
                         2014-04-01 12:00:00         12
                         2015-04-01 12:00:00         12
                         2011-04-01 12:00:00          6
                         2012-04-01 12:00:00          6
                         2013-04-01 12:00:00          6
                         2010-04-01 12:00:00          3
                         2009-04-01 12:00:00          1
dtype: int64

##### requests

In [25]:
print(r4_before.drop('updated_at', axis=1).compare(r4_after.drop('updated_at', axis=1)).shape)
r4_before.drop('updated_at', axis=1).compare(r4_after.drop('updated_at', axis=1)).fillna('').value_counts()

(187, 2)


(submission_date, self)  (submission_date, other)
                         2019-04-01 12:00:00         32
                         2020-04-01 12:00:00         32
                         2018-04-01 12:00:00         23
                         2021-04-01 12:00:00         22
                         2016-04-01 12:00:00         16
                         2017-04-01 12:00:00         16
                         2014-04-01 12:00:00         12
                         2015-04-01 12:00:00         12
                         2011-04-01 12:00:00          6
                         2012-04-01 12:00:00          6
                         2013-04-01 12:00:00          6
                         2010-04-01 12:00:00          3
                         2009-04-01 12:00:00          1
dtype: int64

In [26]:
# same problem statuses
print(r4_after.drop('updated_at', axis=1).compare(r4_after_save.drop('updated_at', axis=1)).shape)
r4_after.drop('updated_at', axis=1).compare(r4_after_save.drop('updated_at', axis=1)).fillna('').value_counts()

(5, 2)


(status, self)  (status, other)
70.0            41.0               3
20.0            30.0               1
41.0            70.0               1
dtype: int64

### Attempt 5: Final Confirmation (v2 dmapps_test2)

In [27]:
requests5 = 'dmapps_test2_v2_csas2_csasrequest.csv'
tor5 = 'dmapps_test2_v2_csas2_termsofreference.csv'

r5_before = pd.read_csv('before/'+requests5)
r5_after = pd.read_csv('after/'+requests5)
r5_after_save = pd.read_csv('after_saving/'+requests5)
tor5_before = pd.read_csv('before/'+tor5)
tor5_after = pd.read_csv('after/'+tor5)
tor5_after_save = pd.read_csv('after_saving/'+tor5)

##### tors

In [28]:
print(tor5_before.drop('updated_at', axis=1).compare(tor5_after.drop('updated_at', axis=1)).shape)
tor5_before.drop('updated_at', axis=1).compare(tor5_after.drop('updated_at', axis=1)).fillna('').value_counts()

(207, 2)


(submission_date, self)  (submission_date, other)
                         2019-04-01 12:00:00         32
                         2020-04-01 12:00:00         32
                         2021-04-01 12:00:00         28
                         2018-04-01 12:00:00         23
                         2016-04-01 12:00:00         16
                         2017-04-01 12:00:00         16
                         2022-04-01 12:00:00         14
                         2014-04-01 12:00:00         12
                         2015-04-01 12:00:00         12
                         2011-04-01 12:00:00          6
                         2012-04-01 12:00:00          6
                         2013-04-01 12:00:00          6
                         2010-04-01 12:00:00          3
                         2009-04-01 12:00:00          1
dtype: int64

In [29]:
print(tor5_before.drop('updated_at', axis=1).compare(tor5_after_save.drop('updated_at', axis=1)).shape)
tor5_before.drop('updated_at', axis=1).compare(tor5_after_save.drop('updated_at', axis=1)).fillna('').value_counts()

(207, 2)


(submission_date, self)  (submission_date, other)
                         2019-04-01 12:00:00         32
                         2020-04-01 12:00:00         32
                         2021-04-01 12:00:00         28
                         2018-04-01 12:00:00         23
                         2016-04-01 12:00:00         16
                         2017-04-01 12:00:00         16
                         2022-04-01 12:00:00         14
                         2014-04-01 12:00:00         12
                         2015-04-01 12:00:00         12
                         2011-04-01 12:00:00          6
                         2012-04-01 12:00:00          6
                         2013-04-01 12:00:00          6
                         2010-04-01 12:00:00          3
                         2009-04-01 12:00:00          1
dtype: int64

##### requests

In [30]:
print(r5_before.drop('updated_at', axis=1).compare(r5_after.drop('updated_at', axis=1)).shape)
r5_before.drop('updated_at', axis=1).compare(r5_after.drop('updated_at', axis=1)).fillna('').value_counts()

(187, 2)


(submission_date, self)  (submission_date, other)
                         2019-04-01 12:00:00         32
                         2020-04-01 12:00:00         32
                         2018-04-01 12:00:00         23
                         2021-04-01 12:00:00         22
                         2016-04-01 12:00:00         16
                         2017-04-01 12:00:00         16
                         2014-04-01 12:00:00         12
                         2015-04-01 12:00:00         12
                         2011-04-01 12:00:00          6
                         2012-04-01 12:00:00          6
                         2013-04-01 12:00:00          6
                         2010-04-01 12:00:00          3
                         2009-04-01 12:00:00          1
dtype: int64

In [31]:
# same problem statuses
print(r5_after.drop('updated_at', axis=1).compare(r5_after_save.drop('updated_at', axis=1)).shape)
r5_after.drop('updated_at', axis=1).compare(r5_after_save.drop('updated_at', axis=1)).fillna('').value_counts()

(5, 2)


(status, self)  (status, other)
70.0            41.0               3
20.0            30.0               1
41.0            70.0               1
dtype: int64

##### compare year quantities - check FY
* they look pretty close for most years. 2021 and 2022 being the exceptions

In [32]:
pd.concat([
    pd.DataFrame(
        r5_before.drop('updated_at', axis=1).compare(r5_after.drop('updated_at', axis=1)).fillna(0).value_counts(),
        columns=['requests']
    ),
    pd.DataFrame(
        tor5_before.drop('updated_at', axis=1).compare(tor5_after.drop('updated_at', axis=1)).fillna(0).value_counts(),
        columns=['tors']
    )
], axis=1).fillna(0).astype(int).sort_index(ascending=False).reset_index().iloc[:, 1:].rename(columns={('submission_date', 'other'): 'date'})

,date,requests,tors
0,2022-04-01 12:00:00,0,14
1,2021-04-01 12:00:00,22,28
2,2020-04-01 12:00:00,32,32
3,2019-04-01 12:00:00,32,32
4,2018-04-01 12:00:00,23,23
5,2017-04-01 12:00:00,16,16
6,2016-04-01 12:00:00,16,16
7,2015-04-01 12:00:00,12,12
8,2014-04-01 12:00:00,12,12
9,2013-04-01 12:00:00,6,6


### what is the deal with those 2022 and 2021 discrepancies?

In [33]:
r_col_idx = [0, 5, 18, 19, 27, 28]
t_col_idx = [0, 5, 10, 15, 16, 17, 18]

In [34]:
# requests
df_r = r5_after[r5_before.submission_date != r5_after.submission_date].dropna(subset=['submission_date'])
request_request_id = [x for x in df_r.id]

In [35]:
# tor
process_request_table = pd.read_csv('after_saving/dmapps_test2_v2_csas2_process_csas_requests.csv')
df_t = tor5_after[(tor5_before.submission_date != tor5_after.submission_date)].dropna(subset=['submission_date'])

tor_process_id = [x for x in df_t.process_id]
tor_request_id = [x for x in process_request_table[process_request_table.process_id.isin(tor_process_id)].csasrequest_id]

In [36]:
# not in tor
r_id_not_in_tor = sorted([x for x in request_request_id if x not in tor_request_id])

# only in tor
r_id_only_in_tor = sorted([x for x in tor_request_id if x not in request_request_id])

# both
r_id_in_both = sorted([x for x in request_request_id if x in tor_request_id])

#### the requests with the status issues:

In [37]:
r5_before[r5_before.status != r5_after_save.status].iloc[:, r_col_idx]

,id,title,status,submission_date,advice_fiscal_year_id,fiscal_year_id
10,12,Science Response - Striped bass SGSL,70,2021-04-28 10:42:19.913898,2021,2022
214,252,2018SAL07 Fraser Sockeye Post Season Run Size ...,70,2021-12-16 12:00:00,2023,2022
256,295,2018SAR12 Recovery Potential Assessment (RPA) ...,70,2021-12-16 12:00:00,2022,2022
766,904,Updated Biological Reference Points for Gasper...,41,2022-10-27 13:48:46.212584,2024,2023
791,930,Assessment of fishery and non-fishery interact...,20,2022-10-26 17:39:16.478281,2026,2023


In [38]:
# how many are in requests, but not in tor?
len(r_id_not_in_tor)

5

In [39]:
# how many are in tor, but not in requests?
len(r_id_only_in_tor)

25

In [40]:
# both
len(r_id_in_both)

182

In [41]:
# seems to add up
182 + 5, 182 + 25

(187, 207)

##### in both

In [42]:
# # in both 
# (commented out because uninteresting)
# r5_before[r5_before.id.isin(r_id_in_both)].iloc[:, r_col_idx]
# tor5_before[tor5_before.process_id.isin([x for x in process_request_table[process_request_table.csasrequest_id.isin(r_id_in_both)].process_id])].iloc[:, t_col_idx]

#### in requests, not in tor

In [43]:
r5_before[r5_before.id.isin(r_id_not_in_tor)].iloc[:, r_col_idx]

,id,title,status,submission_date,advice_fiscal_year_id,fiscal_year_id
68,76,3NOPs4VWX+5 Atlantic Halibut Stock update report,70,NaN,2022,2022
69,77,4X5Y Cod Assessment and New Three-year Project...,99,NaN,2022,2022
74,83,4VWX Silver Hake Assessment,70,NaN,2022,2022
75,84,4X Winter Flounder Stock Assessment,70,NaN,2022,2022
104,114,Application of the National Vulnerability Fram...,70,NaN,2022,2022


In [44]:
# not in tor - these have been submitted or are draft, so look fine
tor5_before[tor5_before.process_id.isin([x for x in process_request_table[process_request_table.csasrequest_id.isin(r_id_not_in_tor)].process_id])].iloc[:, t_col_idx]

,id,process_id,meeting_id,posting_notification_date,posting_request_date,status,submission_date
9,14,25,371.0,NaN,NaN,10,NaN
17,26,35,54.0,NaN,NaN,10,NaN
18,27,36,56.0,NaN,NaN,10,NaN
24,33,42,68.0,NaN,2021-12-22 20:33:22.638798,40,2021-12-22 20:33:22.222174
59,74,34,NaN,NaN,NaN,10,NaN


#### in tor, not in requests

In [45]:
# all have submission dates, so makes sense that they weren't included
r5_before[r5_before.id.isin(r_id_only_in_tor)].iloc[:, r_col_idx]

,id,title,status,submission_date,advice_fiscal_year_id,fiscal_year_id
82,91,Review of at-sea data collection for Lobster F...,70,2021-05-05 01:31:07.431430,2022,2022
98,108,Response on Commercial Viability of Offshore W...,70,2021-05-05 01:31:12.462756,2022,2022
111,122,Offshore Scallop GBa and BBn stock assessment ...,80,2021-11-12 13:44:27.379352,2023,2022
172,204,2020FFHPP05 Review of external geospatial mapp...,80,2021-12-16 12:00:00,2023,2022
182,216,2015GRF04 Coastwide assessment of Canary Rockf...,70,2021-07-09 17:14:53.832179,2022,2022
188,222,2019GRF03 Coastwide Assessment of Arrowtooth ...,70,2021-12-16 12:00:00,2022,2022
194,228,2017INV01 Review of Sea Cucumber (Apostichopus...,80,2021-12-16 12:00:00,2022,2022
204,242,2018SAL05 Review of Escapement Goals for Nass ...,70,2021-12-16 12:00:00,2022,2022
234,273,2019SCI08 Recommendations on Multispecies Bent...,70,2021-12-16 12:00:00,2023,2022
292,336,Science advice on guidance for limit reference...,70,2021-08-26 13:26:39.973821,2023,2022


In [46]:
# all of these needed to be fixed, so this looks correct
# BUT, why are ToR without submission dates connected to requests with submission_dates?
tor5_before[tor5_before.process_id.isin([x for x in process_request_table[process_request_table.csasrequest_id.isin(r_id_only_in_tor)].process_id])].iloc[:, t_col_idx]

,id,process_id,meeting_id,posting_notification_date,posting_request_date,status,submission_date
31,40,49,82.0,2023-01-27 16:13:03.827931,NaN,50,NaN
51,60,69,123.0,2023-01-27 15:54:22.721781,NaN,50,NaN
64,83,84,NaN,2023-01-27 16:15:46.300687,NaN,50,NaN
80,103,111,148.0,2023-01-27 16:23:19.365668,2022-01-06 19:32:54.952369,50,NaN
83,106,114,224.0,2023-01-27 16:07:14.774664,NaN,50,NaN
106,137,145,322.0,2023-01-27 16:16:47.813267,NaN,50,NaN
116,149,157,211.0,2023-01-27 16:08:00.621510,NaN,50,NaN
118,152,160,190.0,2023-01-27 16:20:30.746265,NaN,50,NaN
124,162,170,NaN,2023-01-27 16:22:08.813970,NaN,50,NaN
126,166,174,194.0,2023-01-27 16:27:15.386274,NaN,50,NaN


from SQL:
```SQL
SELECT 
	csas2_csasrequest.id AS r_id, csas2_termsofreference.id AS t_id, csas2_termsofreference.process_id AS p_id, 
	csas2_termsofreference.meeting_id AS m_id, csas2_csasrequest.title AS r_title, 
	csas2_csasrequest.status AS r_status, DATE(csas2_csasrequest.submission_date) AS r_submission_date, 
	DATE(csas2_csasrequest.updated_at) AS r_updated, DATE(csas2_termsofreference.updated_at) AS t_updated,  
	csas2_csasrequest.advice_fiscal_year_id AS r_AFY, csas2_csasrequest.fiscal_year_id AS r_FY, 
	DATE(csas2_termsofreference.posting_notification_date) AS posting_notfication_date, 
	DATE(csas2_termsofreference.posting_request_date) AS posting_request_date, 
	csas2_termsofreference.status AS t_status, DATE(csas2_termsofreference.submission_date) AS t_submission_date 
FROM csas2_termsofreference
JOIN csas2_process ON csas2_termsofreference.process_id = csas2_process.id
JOIN csas2_process_csas_requests ON csas2_process_csas_requests.process_id = csas2_process.id
JOIN csas2_csasrequest ON csas2_csasrequest.id = csas2_process_csas_requests.csasrequest_id
WHERE csas2_csasrequest.status >= 20 AND csas2_termsofreference.status >=20 
	AND csas2_csasrequest.submission_date IS NOT NULL AND csas2_termsofreference.submission_date IS NULL
```

In [47]:
# these don't look like imports. 
# there may be a way to submit a tor without a submission date
pd.read_csv('after_saving/only_in_tor.csv').sort_values('r_title')

,r_id,t_id,p_id,m_id,r_title,r_status,r_submission_date,r_updated,t_updated,r_AFY,r_FY,posting_notfication_date,posting_request_date,t_status,t_submission_date
8,216,162,170,NaN,2015GRF04 Coastwide assessment of Canary Rockf...,70,2021-07-09,2023-06-01,2023-01-27,2022,2022,2023-01-27,NaN,50,NaN
13,773,193,201,205.0,2016OCN03 Guidance on the development of a Mon...,70,2022-02-17,2023-06-01,2023-01-27,2023,2022,2023-01-27,NaN,50,NaN
10,228,173,181,197.0,2017INV01 Review of Sea Cucumber (Apostichopus...,80,2021-12-16,2023-06-01,2023-01-27,2022,2022,2023-01-27,NaN,50,NaN
6,242,149,157,211.0,2018SAL05 Review of Escapement Goals for Nass ...,70,2021-12-16,2023-06-01,2023-01-27,2022,2022,2023-01-27,NaN,50,NaN
9,222,166,174,194.0,2019GRF03 Coastwide Assessment of Arrowtooth ...,70,2021-12-16,2023-01-27,2023-01-27,2022,2022,2023-01-27,NaN,50,NaN
16,273,223,231,218.0,2019SCI08 Recommendations on Multispecies Bent...,70,2021-12-16,2023-06-01,2023-01-27,2023,2022,2023-01-27,NaN,50,NaN
7,204,152,160,190.0,2020FFHPP05 Review of external geospatial mapp...,80,2021-12-16,2023-05-26,2023-01-27,2023,2022,2023-01-27,NaN,50,NaN
12,771,191,199,203.0,2021FFHPP10 - 2022 Update to the application o...,80,2022-02-17,2023-06-01,2023-01-27,2023,2022,2023-01-27,NaN,50,NaN
11,699,190,198,202.0,"2021PEL01 2022/2023 Herring Stock Assessment, ...",80,2021-12-16,2023-05-26,2023-01-27,2023,2022,2023-01-27,NaN,50,NaN
15,702,201,209,210.0,2021PEL04 2022/23 Pacific sardine harvest advi...,80,2021-12-16,2023-05-26,2023-01-27,2022,2022,2023-01-27,NaN,50,NaN


### Investigate the Codebase to check how this is possible:
* all currently messed up tor have tor.status == 50
  * can we confirm this is the only status to worry about?
  * how can we reproduce this?
* **Resuming a review may leave the submission date null**
  * CSASRequestViewSet and ToRViewSet
  * post(): qp.get("resume_review")
  * request_detail\main.html: resumeReview()
    * can only click button if request.status === 25
  * tor\main.html: resumeReview()
    * can only click button if tor.status === 30
* **Modifying the status using the API may will may not update submission_date**
  * def post() from api.views.CSASRequestViewSet
    * "api/csas/requests"
      * resumeReview:
        * OK per above
      * submitCurrentReviewerForm: 
        * look at extended html files
          * csas2/templates/csas2/request_detail/_reviewer_form.html
            * ?approval_seeker=true
            * API approval seeker calls utils.request_approval_seeker(csas_request, self.request)
              * only executes when status is 20
      * withdrawRequest: 
        * "(canModify || isClient) && request.status ===  42"
        * or (2 possible button locations)
        * "canWithdraw" = this.currentUser.can_withdraw && this.currentUser.can_withdraw.is_allowed
  * def post() from api.views.ToRViewSet
    * "api/csas/tors"
      * resumeReview:
        * OK per above
      * postToR
        * "tor.status === 35 && canRequestPost.allowed"
      * submitCurrentReviewerForm: 
        * look at extended html files
          * csas2/templates/csas2/tor/_reviewer_form.html
            * ?approval_seeker=true
            * API approval seeker calls utils.tor_approval_seeker(tor, request)
              * only executes when status is 20
* **models.ToRReviewer.update_parent_status_on_changes_requested() sets tor.status = 30**
  * if models.GenericReviewer.save() with self.decision == 2
    * RequestReviewer(GenericReviewer) makes r.status = 25
    * ToRReviewer(GenericReviewer) makes tor.status = 30
  * when is RequestReviewer or ToRReviewer saved with a decision of 2?
    * 2 is "Return to client"
* **utils.tor_approval_seeker can set tor.status**
  * post() ?approval_seeker or ?resume_review
    * ok, per above
  * torreviewer perform_destroy()

#### How to submit a ToR without submission date:
- submit, review request changes
  - 2 tabs probably required?
    - unsubmit / resume the review (without refreshing) 
- sets status to approved/submitted with no submission date
- after that date stays blank for any updates to the status
- can anything bad happen now?
  - clicking submit can break it and send notification emails